Scripts

# Segment 1 (October 5–October 19)

## API Keys 1 & 2: General election news.

"2024 Presidential election"
"US election AND Trump AND Harris"
"Biden administration AND 2024 election"


## API Key 3: Candidate-specific news.

"Donald Trump AND 2024 election"
"Kamala Harris AND 2024 election"

---
## Script Key 1

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = "your_newsapi_key_1"  # Replace with API key 1
BASE_URL = "https://newsapi.org/v2/everything"

def fetch_articles(query, from_date, to_date, max_requests=100):
    articles = []
    for page in range(1, max_requests + 1):
        params = {
            "q": query,
            "from": from_date,
            "to": to_date,
            "language": "en",
            "pageSize": 100,
            "page": page,
            "apiKey": API_KEY
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            articles.extend(data["articles"])
            if len(data["articles"]) < 100:
                break
        else:
            print(f"Error: {response.status_code}")
            break
        time.sleep(1)
    return articles

# Define date range for Segment 1
start_date = datetime(2024, 10, 5)
end_date = datetime(2024, 10, 19)

from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

print(f"Fetching articles from {from_date} to {to_date}")

queries = ["2024 Presidential election", "US election AND Trump AND Harris", "Biden administration AND 2024 election"]
all_articles = []

# Fetch articles for each query
for query in queries:
    all_articles.extend(fetch_articles(query, from_date, to_date))

df = pd.DataFrame(all_articles)
file_name = f"general_{from_date}_to_{to_date}.csv"
df.to_csv(file_name, index=False)

print(f"Saved {len(all_articles)} articles for General Election News.")


---
# Script key 2

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = "your_newsapi_key_2"  # Replace with API key 2
BASE_URL = "https://newsapi.org/v2/everything"

def fetch_articles(query, from_date, to_date, max_requests=100):
    articles = []
    for page in range(1, max_requests + 1):
        params = {
            "q": query,
            "from": from_date,
            "to": to_date,
            "language": "en",
            "pageSize": 100,
            "page": page,
            "apiKey": API_KEY
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            articles.extend(data["articles"])
            if len(data["articles"]) < 100:
                break
        else:
            print(f"Error: {response.status_code}")
            break
        time.sleep(1)
    return articles

# Define date range for Segment 1
start_date = datetime(2024, 10, 5)
end_date = datetime(2024, 10, 19)

from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

print(f"Fetching articles from {from_date} to {to_date}")

queries = ["2024 Presidential election", "US election AND Trump AND Harris", "Biden administration AND 2024 election"]
all_articles = []

# Fetch articles for each query
for query in queries:
    all_articles.extend(fetch_articles(query, from_date, to_date))

df = pd.DataFrame(all_articles)
file_name = f"general_{from_date}_to_{to_date}.csv"
df.to_csv(file_name, index=False)

print(f"Saved {len(all_articles)} articles for General Election News.")


---
# Script key 3

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = "your_newsapi_key_3"  # Replace with API key 3
BASE_URL = "https://newsapi.org/v2/everything"

def fetch_articles(query, from_date, to_date, max_requests=100):
    articles = []
    for page in range(1, max_requests + 1):
        params = {
            "q": query,
            "from": from_date,
            "to": to_date,
            "language": "en",
            "pageSize": 100,
            "page": page,
            "apiKey": API_KEY
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            articles.extend(data["articles"])
            if len(data["articles"]) < 100:
                break
        else:
            print(f"Error: {response.status_code}")
            break
        time.sleep(1)
    return articles

# Define date range for Segment 1
start_date = datetime(2024, 10, 5)
end_date = datetime(2024, 10, 19)

from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

print(f"Fetching articles from {from_date} to {to_date}")

queries = ["Donald Trump AND 2024 election", "Kamala Harris AND 2024 election"]
all_articles = []

# Fetch articles for each query
for query in queries:
    all_articles.extend(fetch_articles(query, from_date, to_date))

df = pd.DataFrame(all_articles)
file_name = f"general_{from_date}_to_{to_date}.csv"
df.to_csv(file_name, index=False)

print(f"Saved {len(all_articles)} articles for General Election News.")


---
# script 4

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = "your_newsapi_key_4"  # Replace with API key 4
BASE_URL = "https://newsapi.org/v2/everything"

def fetch_articles(query, from_date, to_date, max_requests=100):
    articles = []
    for page in range(1, max_requests + 1):
        params = {
            "q": query,
            "from": from_date,
            "to": to_date,
            "language": "en",
            "pageSize": 100,
            "page": page,
            "apiKey": API_KEY
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            articles.extend(data["articles"])
            if len(data["articles"]) < 100:
                break
        else:
            print(f"Error: {response.status_code}")
            break
        time.sleep(1)
    return articles

# Define date range for Segment 1
start_date = datetime(2024, 10, 5)
end_date = datetime(2024, 10, 19)

from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

print(f"Fetching articles from {from_date} to {to_date}")

queries = ["Presidential debate AND 2024", "Trump rally AND 2024", "Harris speech AND 2024"]
all_articles = []

# Fetch articles for each query
for query in queries:
    all_articles.extend(fetch_articles(query, from_date, to_date))

df = pd.DataFrame(all_articles)
file_name = f"general_{from_date}_to_{to_date}.csv"
df.to_csv(file_name, index=False)

print(f"Saved {len(all_articles)} articles for General Election News.")


---
# Script 5

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = "your_newsapi_key_5"  # Replace with API key 5
BASE_URL = "https://newsapi.org/v2/everything"

def fetch_articles(query, from_date, to_date, max_requests=100):
    articles = []
    for page in range(1, max_requests + 1):
        params = {
            "q": query,
            "from": from_date,
            "to": to_date,
            "language": "en",
            "pageSize": 100,
            "page": page,
            "apiKey": API_KEY
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            articles.extend(data["articles"])
            if len(data["articles"]) < 100:
                break
        else:
            print(f"Error: {response.status_code}")
            break
        time.sleep(1)
    return articles

# Define date range for Segment 1
start_date = datetime(2024, 10, 5)
end_date = datetime(2024, 10, 19)

from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

print(f"Fetching articles from {from_date} to {to_date}")

queries = ["Presidential debate AND 2024", "Trump rally AND 2024", "Harris speech AND 2024"]
all_articles = []

# Fetch articles for each query
for query in queries:
    all_articles.extend(fetch_articles(query, from_date, to_date))

df = pd.DataFrame(all_articles)
file_name = f"general_{from_date}_to_{to_date}.csv"
df.to_csv(file_name, index=False)

print(f"Saved {len(all_articles)} articles for General Election News.")


---
# Script 6

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

API_KEY = "your_newsapi_key_6"  # Replace with API key 6
BASE_URL = "https://newsapi.org/v2/everything"

def fetch_articles(query, from_date, to_date, max_requests=100):
    articles = []
    for page in range(1, max_requests + 1):
        params = {
            "q": query,
            "from": from_date,
            "to": to_date,
            "language": "en",
            "pageSize": 100,
            "page": page,
            "apiKey": API_KEY
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            articles.extend(data["articles"])
            if len(data["articles"]) < 100:
                break
        else:
            print(f"Error: {response.status_code}")
            break
        time.sleep(1)
    return articles

# Define date range for Segment 1
start_date = datetime(2024, 10, 5)
end_date = datetime(2024, 10, 19)

from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

print(f"Fetching articles from {from_date} to {to_date}")

queries = ["Campaign financing AND 2024 election", "Voter turnout AND 2024"]
all_articles = []

# Fetch articles for each query
for query in queries:
    all_articles.extend(fetch_articles(query, from_date, to_date))

df = pd.DataFrame(all_articles)
file_name = f"general_{from_date}_to_{to_date}.csv"
df.to_csv(file_name, index=False)

print(f"Saved {len(all_articles)} articles for General Election News.")


Combine datasets - not sure

In [ ]:
import glob
import pandas as pd

all_files = glob.glob("*.csv")
combined_df = pd.concat([pd.read_csv(file) for file in all_files])
combined_df.drop_duplicates(subset="title", inplace=True)
combined_df.to_csv("final_combined_data.csv", index=False)

print(f"Saved combined dataset with {len(combined_df)} unique articles.")
